# Part 3 - Stage 2 Data Manipulation

The purpose of this library is to manipulate stage 2 data into useable training data. 

The inputs you need for this are:
/data/MNCAATourneySeeds.csv
/data/MRegularSeasonDetailedResults.csv

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pkg_resources

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

C:\Users\C176616\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\C176616\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-

In [20]:
cwd = os.getcwd()
cwd

'C:\\Users\\C176616\\Documents\\GitHub\\MarchMadness'

## Data Manipulation

### Regular Season Data Analysis

In [30]:
# Existing code from data manipulation section. Only run if needed. 
cwd = os.getcwd()

# tourney_cresults = pd.read_csv(cwd + '/data/MNCAATourneyCompactResults.csv')
# seeds = pd.read_csv(cwd + '/data/MNCAATourneySeeds.csv')
seeds = pd.read_csv(cwd + '/data_stage2/MNCAATourneySeeds.csv')
season_dresults = pd.read_csv(cwd +'/data_stage2/MRegularSeasonDetailedResults.csv')

targetYear = 2003
# tourney_cresults = tourney_cresults.loc[tourney_cresults['Season'] >= targetYear]

training_set = pd.read_csv("training_set.csv")
record = pd.read_csv('record.csv')

seeds['Seed'] =  pd.to_numeric(seeds['Seed'].str[1:3], downcast='integer',errors='coerce')

def delta_seed(row):
    cond = (seeds['Season'] == row['Season'])
    return seeds[cond & (seeds['TeamID'] == row['Team1'])]['Seed'].iloc[0] - seeds[cond & (seeds['TeamID'] == row['Team2'])]['Seed'].iloc[0]

# function to, given a row, calculate what the difference between the two seeds was. 
#Function to look up 
def delta_winPct(row):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1]['wins']/record[cond1]['games']).mean() - (record[cond2]['wins']/record[cond2]['games']).mean()

def get_points_against(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'LScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'WScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_points_for(row):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'WScore'
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'LScore'
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum() 
    return retVal

def get_remaining_stats(row, field):
    wcond = (dfW['Season'] == row['Season']) & (dfW['WTeamID'] == row['WTeamID']) 
    fld1 = 'W' + field
    lcond = (dfL['Season'] == row['Season']) & (dfL['LTeamID'] == row['WTeamID']) 
    fld2 = 'L'+ field
    retVal = dfW[wcond][fld1].sum()
    if len(dfL[lcond][fld2]) > 0:
        retVal = retVal + dfL[lcond][fld2].sum()
    return retVal

def delta_stat(row, field):
    cond1 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team1'])
    cond2 = (record['Season'] == row['Season']) & (record['WTeamID'] == row['Team2'])
    return (record[cond1][field]/record[cond1]['games']).mean() - (record[cond2][field]/record[cond2]['games']).mean()

Ok, so now we have a trained model. Next we need to find sumission data.

The kaggle competition provides a sample submission.csv file that contains a matchup ID, and a default prediction value. 

In [31]:
sub = pd.read_csv(cwd + '/data_stage2/MSampleSubmissionStage2.csv')
sub

,ID,Pred
0,2022_1103_1104,0.5
1,2022_1103_1112,0.5
2,2022_1103_1116,0.5
3,2022_1103_1120,0.5
4,2022_1103_1124,0.5
...,...,...
2273,2022_1458_1461,0.5
2274,2022_1458_1463,0.5
2275,2022_1460_1461,0.5
2276,2022_1460_1463,0.5


In [27]:
sub = sub.replace(r'2022', '2023', regex=True)

Split this string into Team IDs and year

In [28]:
sub['Season'], sub['Team1'], sub['Team2'] = sub['ID'].str.split('_').str
sub[['Season', 'Team1', 'Team2']] = sub[['Season', 'Team1', 'Team2']].apply(pd.to_numeric)
sub

C:\Users\C176616\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """Entry point for launching an IPython kernel.


,ID,Pred,Season,Team1,Team2,deltaSeed
0,2023_1103_1104,0.5,2023,1103,1104,7
1,2023_1103_1112,0.5,2023,1103,1112,12
2,2023_1103_1116,0.5,2023,1103,1116,9
3,2023_1103_1120,0.5,2023,1103,1120,11
4,2023_1103_1124,0.5,2023,1103,1124,12
...,...,...,...,...,...,...
2273,2023_1458_1461,0.5,2023,1458,1461,-9
2274,2023_1458_1463,0.5,2023,1458,1463,-11
2275,2023_1460_1461,0.5,2023,1460,1461,4
2276,2023_1460_1463,0.5,2023,1460,1463,2


Calculate the deltaSeed and deltaWinPct features

In [29]:
sub['deltaSeed'] = sub.apply(delta_seed,axis=1)
# sub['deltaMO'] = sub.apply(delta_ord,axis=1)


IndexError: single positional indexer is out-of-bounds

In [26]:
sub

,ID,Pred,Season,Team1,Team2,deltaSeed
0,2022_1103_1104,0.5,2022,1103,1104,7
1,2022_1103_1112,0.5,2022,1103,1112,12
2,2022_1103_1116,0.5,2022,1103,1116,9
3,2022_1103_1120,0.5,2022,1103,1120,11
4,2022_1103_1124,0.5,2022,1103,1124,12
...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,-9
2274,2022_1458_1463,0.5,2022,1458,1463,-11
2275,2022_1460_1461,0.5,2022,1460,1461,4
2276,2022_1460_1463,0.5,2022,1460,1463,2


In [ ]:
sub['deltaWinPct'] = sub.apply(delta_winPct,axis=1)
sub

Now, caluclate the rest of our stats. This will take a while.

In [7]:
# cut to slides
rawCols = ['Seed', 'WinPct', 'PointsFor','PointsAgainst','FGM','FGA','FGM3','FGA3','FTM','FTA','OR','DR','Ast','TO','Stl','Blk','PF']

for rawCol in rawCols:
    print("Processing",rawCol)
    sub['delta' + rawCol] = sub.apply(delta_stat,args=(rawCol,),axis=1)

Processing Seed


KeyError: 'Seed'

In [8]:
sub.to_csv("training_set_stage2.csv", index=False)
sub

,ID,Pred,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,2022_1103_1104,0.5,2022,1103,1104,7,0.115927,-10.678427,-12.244960,-4.075605,...,-8.416331,-0.955645,-0.223790,-3.754032,-1.234879,-3.108871,-3.255040,-1.516129,-1.621976,-2.626008
1,2022_1103_1112,0.5,2022,1103,1112,12,-0.202087,-15.268501,-3.368121,-6.860531,...,-0.234345,-1.492410,-0.720114,-2.570209,-4.869070,-8.395636,-2.102467,-1.222011,-2.609108,-0.502846
2,2022_1103_1116,0.5,2022,1103,1116,9,-0.047898,-7.649071,-4.384164,-3.055718,...,0.919844,-2.883675,-2.130987,-1.826002,-2.000978,-2.574780,-1.312805,-2.243402,-1.085044,-0.880743
3,2022_1103_1120,0.5,2022,1103,1120,11,-0.134073,-9.428427,-2.869960,-4.356855,...,-3.697581,-0.299395,0.744960,-2.347782,-2.234879,-3.015121,-0.817540,-3.297379,-4.746976,-2.563508
4,2022_1103_1124,0.5,2022,1103,1124,12,-0.102823,-7.209677,0.536290,-4.638105,...,-1.291331,2.294355,3.463710,-3.097782,0.733871,-4.327621,-1.161290,-3.328629,-0.309476,0.123992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,-9,0.024194,-1.165323,0.860887,0.156250,...,-1.683468,-0.559476,-1.218750,0.996976,-1.804435,0.125000,-2.393145,1.509073,0.743952,1.686492
2274,2022_1458_1463,0.5,2022,1458,1463,-11,0.153504,-1.083426,-3.210234,-0.482759,...,0.163515,0.165740,0.103448,0.385984,-1.293660,-0.827586,-4.235818,-0.709677,-0.365962,-0.618465
2275,2022_1460_1461,0.5,2022,1460,1461,4,-0.143939,3.276515,5.918561,2.580492,...,-3.721591,-0.656250,-1.946023,2.063447,-3.214015,2.912879,1.339015,1.976326,0.846591,-0.766098
2276,2022_1460_1463,0.5,2022,1460,1463,2,-0.014629,3.358412,1.847440,1.941484,...,-1.874608,0.068966,-0.623824,1.452456,-2.703239,1.960293,-0.503657,-0.242424,-0.263323,-3.071055


In [9]:
sub

,ID,Pred,Season,Team1,Team2,deltaSeed,deltaWinPct,deltaPointsFor,deltaPointsAgainst,deltaFGM,...,deltaFGA3,deltaFTM,deltaFTA,deltaOR,deltaDR,deltaAst,deltaTO,deltaStl,deltaBlk,deltaPF
0,2022_1103_1104,0.5,2022,1103,1104,7,0.115927,-10.678427,-12.244960,-4.075605,...,-8.416331,-0.955645,-0.223790,-3.754032,-1.234879,-3.108871,-3.255040,-1.516129,-1.621976,-2.626008
1,2022_1103_1112,0.5,2022,1103,1112,12,-0.202087,-15.268501,-3.368121,-6.860531,...,-0.234345,-1.492410,-0.720114,-2.570209,-4.869070,-8.395636,-2.102467,-1.222011,-2.609108,-0.502846
2,2022_1103_1116,0.5,2022,1103,1116,9,-0.047898,-7.649071,-4.384164,-3.055718,...,0.919844,-2.883675,-2.130987,-1.826002,-2.000978,-2.574780,-1.312805,-2.243402,-1.085044,-0.880743
3,2022_1103_1120,0.5,2022,1103,1120,11,-0.134073,-9.428427,-2.869960,-4.356855,...,-3.697581,-0.299395,0.744960,-2.347782,-2.234879,-3.015121,-0.817540,-3.297379,-4.746976,-2.563508
4,2022_1103_1124,0.5,2022,1103,1124,12,-0.102823,-7.209677,0.536290,-4.638105,...,-1.291331,2.294355,3.463710,-3.097782,0.733871,-4.327621,-1.161290,-3.328629,-0.309476,0.123992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,-9,0.024194,-1.165323,0.860887,0.156250,...,-1.683468,-0.559476,-1.218750,0.996976,-1.804435,0.125000,-2.393145,1.509073,0.743952,1.686492
2274,2022_1458_1463,0.5,2022,1458,1463,-11,0.153504,-1.083426,-3.210234,-0.482759,...,0.163515,0.165740,0.103448,0.385984,-1.293660,-0.827586,-4.235818,-0.709677,-0.365962,-0.618465
2275,2022_1460_1461,0.5,2022,1460,1461,4,-0.143939,3.276515,5.918561,2.580492,...,-3.721591,-0.656250,-1.946023,2.063447,-3.214015,2.912879,1.339015,1.976326,0.846591,-0.766098
2276,2022_1460_1463,0.5,2022,1460,1463,2,-0.014629,3.358412,1.847440,1.941484,...,-1.874608,0.068966,-0.623824,1.452456,-2.703239,1.960293,-0.503657,-0.242424,-0.263323,-3.071055
